# Hierarchical Feature Visualization in a Jupyter Notebook

## Load a Pre-trained CNN Model.
Load a pre-trained CNN model (e.g., VGG16, ResNet50, Inception) using a deep learning framework like Keras or PyTorch.

The CNN Model: VGG19

[VGG19 architecture](https://arxiv.org/abs/1409.1556) consists of 16 consecutive 3x3 convolution layers with max poolings in between, followed by 3 fully connected layers. We are interested in examining input patterns that excite the neurons in different convolutional layers. 

<figure>
<img src="https://raw.githubusercontent.com/mamaj/cnn-featurevis-ece421/master/figs/vgg19_2.png"
     alt="VGG19 architecture"
     style="float: left; margin-right: 5px;"
     height="300px" />
<figcaption>VGG19 architecture</figcaption>

</figure>

We can import a VGG19 model, pretrained on [ImageNet dataset](http://www.image-net.org/), using tensorflow.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load a pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

## Define a Visualization Model.
Create a new model that takes an input image and outputs the feature maps (activations) from selected intermediate layers. This allows you to inspect the features learned by each layer.

In [ ]:
layer_names = [layer.name for layer in model.layers if 'conv' in layer.name or 'pool' in layer.name]
layer_outputs = [model.get_layer(name).output for name in layer_names]
visualization_model = Model(inputs=model.input, outputs=layer_outputs)

## Prepare an Input Image.
Load and preprocess an image for which you want to visualize the features. This typically involves resizing, normalizing, and expanding dimensions to match the model's input requirements.

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = 'coconut.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
# Use canonical preprocessing for VGG16 to improve stability/contrast
img_array = preprocess_input(img_array)

Show the image

In [ ]:
matplotlib.pyplot.imshow(img)
matplotlib.pyplot.show()

## Obtain Feature Maps.
Pass the preprocessed image through the visualization_model to get the feature maps for each selected layer.

In [ ]:
feature_maps = visualization_model.predict(img_array)

## Visualize Feature Maps.
Iterate through the obtained feature maps and plot them. You can visualize individual feature maps or a selection of them to observe the patterns and activations learned at different hierarchical levels.

In [ ]:
import matplotlib.pyplot as plt

max_features_per_layer = None  # set to an int to limit per-layer features

for layer_name, feature_map in zip(layer_names, feature_maps):
    if len(feature_map.shape) == 4:  # Only visualize 2D feature maps
        total_features = feature_map.shape[-1]
        num_features = total_features if max_features_per_layer is None else min(total_features, max_features_per_layer)

        for i in range(num_features):
            x = feature_map[0, :, :, i]
            std = x.std()
            if std < 1e-6:
                x = np.zeros_like(x)
            else:
                x = (x - x.mean()) / std
                x = (x * 64 + 128)
            x = np.clip(x, 0, 255).astype('uint8')

            plt.figure(figsize=(4, 4))
            plt.title(f"{layer_name} — feature {i}")
            plt.imshow(x, cmap='viridis')
            plt.axis('off')
            plt.show()

## Visualize Filters in the Block1_Conv1 and Block5_Conv3

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input

# Utility to convert a preprocessed image tensor back to displayable RGB uint8
def deprocess_image(x: tf.Tensor) -> np.ndarray:
    x = x[0].numpy()
    # BGR mean values used in VGG16 preprocessing
    x[..., 0] += 103.939
    x[..., 1] += 116.779
    x[..., 2] += 123.68
    # Convert BGR to RGB
    x = x[..., ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Perform gradient ascent on the input image to maximize a given filter's activation
def visualize_filter(layer_name: str, filter_index: int, steps: int = 30, step_size: float = 1.0, img_size: int = 224) -> np.ndarray:
    submodel = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    # Start from random noise in pixel space, then preprocess to VGG16 space
    init_img = tf.random.uniform((1, img_size, img_size, 3), minval=0.0, maxval=255.0)
    img = preprocess_input(init_img)
    img = tf.Variable(img)

    for _ in range(steps):
        with tf.GradientTape() as tape:
            tape.watch(img)
            activation = submodel(img)
            loss = tf.reduce_mean(activation[:, :, :, filter_index])
        grads = tape.gradient(loss, img)
        # Normalize gradients for stable updates
        grads = grads / (tf.math.reduce_std(grads) + 1e-8)
        img.assign_add(step_size * grads)

    return deprocess_image(img)

# Visualize filters for specified layers; set max_filters_per_layer=None to run all
layers_to_visualize = ['block1_conv1', 'block5_conv3']
max_filters_per_layer = None  # e.g., set to 16 to limit

for layer_name in layers_to_visualize:
    num_filters = int(model.get_layer(layer_name).output.shape[-1])
    count = num_filters if max_filters_per_layer is None else min(num_filters, max_filters_per_layer)
    for i in range(count):
        img = visualize_filter(layer_name, i, steps=30, step_size=1.0, img_size=224)
        plt.figure(figsize=(3, 3))
        plt.title(f"{layer_name} — filter {i}")
        plt.imshow(img)
        plt.axis('off')
        plt.show()


plot each channel (3x3) for each filter in block5_conv3.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Visualize per-channel 3x3 kernels for each filter in block5_conv3
# NOTE: block5_conv3 has shape (3, 3, 512, 512) => 512 filters, each with 512 input channels
# Plotting all channels for all filters would create 262,144 tiles.
# Use the selectors below to limit for practicality.

layer_name = 'block5_conv3'
w = model.get_layer(layer_name).get_weights()[0]  # (3, 3, Cin, Cout)
kh, kw, cin, cout = w.shape
print(f"{layer_name} kernel weights shape: {w.shape} -> (kh, kw, Cin, Cout)")

# Select which filters (output channels) and input channels to display
# Adjust these as needed. To show all channels for a single filter, set channels_to_plot = range(cin)
filters_to_plot = list(range(0, min(2, cout)))        # e.g., first 2 filters
channels_to_plot = list(range(0, min(16, cin)))       # e.g., first 16 input channels

for f_idx in filters_to_plot:
    num_channels = len(channels_to_plot)
    cols = min(8, num_channels)
    rows = int(np.ceil(num_channels / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 1.2, rows * 1.2))
    axes = np.atleast_2d(axes)
    fig.suptitle(f"{layer_name} — filter {f_idx}: per-channel 3x3 kernels", fontsize=10)

    for i, c_idx in enumerate(channels_to_plot):
        r, c = divmod(i, cols)
        k = w[:, :, c_idx, f_idx]  # (3, 3)
        # Normalize per-channel for visualization
        k_min, k_max = k.min(), k.max()
        k_norm = (k - k_min) / (k_max - k_min + 1e-8)
        axes[r, c].imshow(k_norm, cmap='gray', vmin=0.0, vmax=1.0, interpolation='nearest')
        axes[r, c].set_xticks([])
        axes[r, c].set_yticks([])
        axes[r, c].set_title(str(c_idx), fontsize=8)

    # Hide any unused subplots
    for j in range(num_channels, rows * cols):
        r, c = divmod(j, cols)
        axes[r, c].axis('off')

    plt.tight_layout()
    plt.show()
